In [9]:
# sorry for the jank, going to have to run each code cell one probably before anything will work

import networkx as nx
import matplotlib.pyplot as plt
from itertools import combinations

# Multigraph route?

In [10]:
#probably worse way now that I read some more documentation, but this route exists
cubeQuestionmark = [(1, 2, {"vertices": [3,4]}), (1, 3, {"vertices": [1,3]}), (1, 4, {"vertices": [2,4]}), (1, 5, {"vertices": [1,3]}), 
 (2, 3, {"vertices": [3,5]}), (2, 4, {"vertices": [4,6]}), (2, 6, {"vertices": [5,6]}),
 (3, 5, {"vertices": [1,7]}), (3, 6, {"vertices": [5,7]}),
 (4, 5, {"vertices": [2,8]}), (4, 6, {"vertices": [6,8]}), 
 (5, 6, {"vertices": [7,8]})]
F = nx.Graph()
F.add_edges_from(cubeQuestionmark)
#nx.draw(F)
#plt.show()

In [11]:
#terms
#node = "vertex" in graph, represents face/faset
#edge = connection between nodes (can be multiple, is a multigraph structure)
    #each edge is effectivley 2 "parts" of a true edge, aka the 2 (for now) points that constitute an edge


class BadIdea:
                #Untouched, New,   New-1,   New-2,    Old
    edgeColors = ["black", "red", "orange", "yellow", "green"]
    nodeColors = ["black", "red", "orange", "yellow", "green"]
    def __init__(self):
        self.graph = nx.MultiGraph()
        #these are for the object, not the graph
        self.vertexList = []
        self.edgeList = []
        
    def Draw(self):
        pos = nx.circular_layout(self.graph) #maybe remove need to check what this is
        edges = self.graph.edges(keys=True)
        colors = [self.graph[u][v][k]['badIdeaObj'].color for u,v,k in edges]
        #weights = [G[u][v]['edgeObj'].weight for u,v in edges]
        nx.draw(self.graph, pos, edges=edges, edge_color=colors) # #width=weights)
        
        
        
    #parameters: boolean list with fasets being the rows, and verticies the collumns
    def bool_to_graph(self, boolArray):
        #reset graph?
        self.graph = nx.MultiGraph()
        self.vertexList = []
        self.edgeList = []
        
        #add base nodes
        for i in range(len(boolArray)):
            self.graph.add_node(i)
        
        #Create BadIdeaVertex objects 
        for i in range(len(boolArray[0])):
            self.vertexList.append(BadIdeaVertex(i))
            
        #get edge list from adjacency list
        edgeSet = []
        for vertex in range(len(boolArray[0])):
            vertList = []
            for face in range(len(boolArray)):
                if boolArray[face][vertex]:
                    vertList.append(face)
                    self.vertexList[vertex].connectedFaces.add(face)
                    
            #add every combination of 2 faces that this vertex is connected to, to the list
            for edge in combinations(vertList, 2):
                edgeObj = BadIdeaEdge(0, edge[0], edge[1], self)
                self.edgeList.append(edgeObj)
                key = self.graph.add_edge(edge[0], edge[1])
                edgeObj.set_key(key)
                edgeObj.graph_edge['badIdeaObj'] = edgeObj
                edgeObj.connect_vertex(self.vertexList[vertex])

        #show graph
        self.Draw()
        
        print("Number of nodes:", self.graph.number_of_nodes())
        print("Number of edges:", self.graph.number_of_edges())
        print("Number of verticies:", self.graph.number_of_edges())
        
    #janked into working from my below code. Surely not optimal just wanted to see if it'd even hold up
    #Is not using anything 'unique' to multigraphs or even just yet
    def solve_greedy(self):
        untouched_faces = list(range(self.graph.number_of_nodes()))
        output = [] #vertexes used in solution
        
        solved = False
        while not solved:
            self.Draw()
            print()
            #Calculate the worth of each remaining 
            vertex_values = [ 0 for i in range(len(self.vertexList))]
            for vertex in range(len(self.vertexList)):
                for face in self.vertexList[vertex].connectedFaces:
                    if face in untouched_faces:
                        vertex_values[vertex] += 1
                            
            print("Vertex Values:", vertex_values)
            
            # Find vertex that connects to the most unconnected faces, really unclear, but it should work, 
            # TODO: rework into something more human readable
            max_vertex = [-1,0]
            for index in range(len(vertex_values)):
                if vertex_values[index] > max_vertex[1]:
                    max_vertex = [index, vertex_values[index]]
            
            # Mark that vertex as used, and mark all the faces it touches as touched
            output.append(self.vertexList[max_vertex[0]])
            for face in range(len(untouched_faces)):
                if face in self.vertexList[max_vertex[0]].connectedFaces:
                    untouched_faces[face] = -1
            #color
            for edge in self.vertexList[max_vertex[0]].connectedEdges:
                edge.update_color()
                    
            # Exit if the faces are all touched  
            flag = False
            for i in untouched_faces:
                if i is not -1:
                    flag = True
                    break
                    
            if not flag:
                self.Draw()
                return output
                
        
        
class BadIdeaVertex:
    def __init__(self, __id):
        self.__id__ = __id
        self.connectedEdges = set()
        self.connectedFaces = set()
        
    def faces_to_string(self):
        return str(self.connectedFaces)
         
    @property
    def get_id(self):
        return self.__id__
        
class BadIdeaEdge:
    def __init__(self, color, face1, face2, parent):
        self.__color__ = color
        self.__face1__ = face1
        self.__face2__ = face2
        self.parent = parent
        self.__key__ = None
        self.__vertex__ = None
    
    def set_key(self, key):
        self.__key__ = key
        
    def connect_vertex(self, vertex):
        self.__vertex__ = vertex
        vertex.connectedEdges.add(self)
            
    def update_color(self):
        self.__color__ += 1
        
    @property
    def color(self):
        return self.parent.edgeColors[min(self.__color__, len(self.parent.edgeColors)-1)]
    
    @property
    def full_path(self):
        return (self.face1, self.face2, self.key)
    
    @property
    def graph_edge(self):
        return self.parent.graph[self.face1][self.face2][self.key]
    
    @property
    def face1(self):
        return self.__face1__
    
    @property
    def face2(self):
        return self.__face2__
    
    @property
    def key(self):
        return self.__key__
    
    @property
    def vertex(self):
        return self.__vertex__

In [12]:
print(newGraph.graph.edges())

NameError: name 'newGraph' is not defined

In [ ]:
newGraph = BadIdea()
newGraph.bool_to_graph(int_shape_to_bool(cube))
out = newGraph.solve_greedy()
print("Solution set")
for i in out: print("Vertex ID:", i.get_id, "Connected Faces:", i.faces_to_string())


# Custom route?

In [7]:
cube = [[1,0,0,1,1,0,0,1],
        [1,1,0,0,1,1,0,0],
        [1,1,1,1,0,0,0,0],
        [0,1,1,0,0,1,1,0],
        [0,0,1,1,0,0,1,1],
        [0,0,0,0,1,1,1,1]]

#[print(row) for row in int_shape_to_bool(cube)]

In [8]:
# Helper function to transform 1s to True, 0s to False
def int_shape_to_bool(shape):
    out = []
    for row in shape:
        out.append([])
        for cell in row:
            if cell == 1:
                out[-1].append(True)
            elif cell == 0:
                out[-1].append(False)
    return out